In [1]:
!pip install gensim

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 6.5 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 KB 5.2 MB/s eta 0:00:00


In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
import tensorflow as tf
import pickle
import gensim
from tensorflow.keras.utils import to_categorical



2024-05-29 17:20:17.230832: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-29 17:20:17.637745: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-29 17:20:17.641062: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-29 17:20:19.278224: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


DOWNLOAD WORD2VEC PRE-TRAINED EMBEDDING AT  https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?resourcekey=0-wjGZdNAUop6WykTtMip30g

In [2]:
word2vec_path = 'GoogleNews-vectors-negative300.bin'
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)
print(f'Found {len(word2vec_model.key_to_index)} word vectors.')

Found 3000000 word vectors.


In [3]:
data = pd.read_csv("../preprocessing/all-data-processed-3classes.csv")
data["Clean sentences"] = data["Clean sentences"].astype(str)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(data["Clean sentences"], data["Sentiment"], test_size=0.1, random_state=42)

In [6]:
token = Tokenizer()
token.fit_on_texts(X_train)

In [7]:
vocab = len(token.index_word) + 1
X_train = token.texts_to_sequences(X_train)
X_test = token.texts_to_sequences(X_test)

In [8]:
MAX_SEQUENCE_LENGTH = 47
X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH, padding="post")
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH, padding="post")

In [9]:
y_train_labels = np.array(y_train)  # Convert to NumPy array if not already
y_test_labels = np.array(y_test)  # Convert to NumPy array if not already

y_train = np.where(y_train_labels == -1, 0, y_train_labels)
y_train = np.where(y_train_labels == 0, 1, y_train)
y_train = np.where(y_train_labels == 1, 2, y_train)

y_test = np.where(y_test_labels == -1, 0, y_test_labels)
y_test = np.where(y_test_labels == 0, 1, y_test)
y_test = np.where(y_test_labels == 1, 2, y_test)

y_train = to_categorical(y_train, num_classes=3)
y_test = to_categorical(y_test, num_classes=3)

In [11]:
embedding_dim = 300  # Size of Word2Vec embeddings

# Prepare embedding matrix
embedding_matrix = np.zeros((vocab, embedding_dim))
for word, i in token.word_index.items():
    if i < vocab:
        if word in word2vec_model:
            embedding_vector = word2vec_model[word]
            embedding_matrix[i] = embedding_vector

In [12]:
vec_size = 300  # Word2Vec embedding size
model = Sequential()
model.add(Embedding(vocab, vec_size, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH, trainable=False))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))  # Output layer with softmax activation

model.compile(loss='categorical_crossentropy', optimizer=tf.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])
model.summary()

2024-05-29 17:21:44.829300: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-05-29 17:21:44.831098: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-05-29 17:21:44.832653: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 47, 300)           8724600   
                                                                 
 lstm (LSTM)                 (None, 47, 128)           219648    
                                                                 
 dropout (Dropout)           (None, 47, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 3)                 195       
                                                                 
Total params: 8,993,851
Trainable params: 269,251
Non-tr

2024-05-29 17:21:45.050538: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-05-29 17:21:45.052294: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-05-29 17:21:45.053512: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [13]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model_lstm.h5', save_best_only=True, monitor='val_loss', mode='min')

# Define a learning rate scheduler
def scheduler(epoch, lr):
    if epoch < 5:
        return lr
    else:
        return lr * 0.9

lr_scheduler = LearningRateScheduler(scheduler)

In [14]:
history = model.fit(X_train, y_train, epochs=40, validation_data=(X_test, y_test), batch_size=32, callbacks=[early_stopping, model_checkpoint, lr_scheduler])

Epoch 1/40


2024-05-29 17:21:56.706165: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-05-29 17:21:56.707897: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-05-29 17:21:56.709064: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

896/897 [============================>.] - ETA: 0s - loss: 0.9252 - accuracy: 0.5425

2024-05-29 17:22:56.981124: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-05-29 17:22:56.982616: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-05-29 17:22:56.983625: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

897/897 [==============================] - 63s 67ms/step - loss: 0.9251 - accuracy: 0.5425 - val_loss: 0.8518 - val_accuracy: 0.6119 - lr: 1.0000e-04
Epoch 2/40
897/897 [==============================] - 59s 66ms/step - loss: 0.7280 - accuracy: 0.6949 - val_loss: 0.6903 - val_accuracy: 0.7176 - lr: 1.0000e-04
Epoch 3/40
897/897 [==============================] - 60s 67ms/step - loss: 0.6494 - accuracy: 0.7360 - val_loss: 0.6589 - val_accuracy: 0.7352 - lr: 1.0000e-04
Epoch 4/40
897/897 [==============================] - 60s 67ms/step - loss: 0.6128 - accuracy: 0.7543 - val_loss: 0.6252 - val_accuracy: 0.7531 - lr: 1.0000e-04
Epoch 5/40
897/897 [==============================] - 59s 66ms/step - loss: 0.5787 - accuracy: 0.7732 - val_loss: 0.6143 - val_accuracy: 0.7540 - lr: 1.0000e-04
Epoch 6/40
897/897 [==============================] - 59s 65ms/step - loss: 0.5406 - accuracy: 0.7913 - val_loss: 0.5850 - val_accuracy: 0.7719 - lr: 9.0000e-05
Epoch 7/40
897/897 [=========================

In [15]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

# Get predicted probabilities
predictions = model.predict(X_test)

# Print some example predictions
print("Predicted probabilities:", predictions[:10])
print("Actual labels:", y_test[:10])

# Convert one-hot encoded y_test back to class labels for comparison
true_classes = np.argmax(y_test, axis=1)

# Create a DataFrame to compare predictions with actual values
results_df = pd.DataFrame({
    'Text': [" ".join([token.index_word.get(idx, "") for idx in x if idx != 0]) for x in X_test],
    'Actual Class': true_classes,
    'Predicted Class': np.argmax(predictions, axis=1),
    'Predicted Probabilities': list(predictions)
})

# Show the DataFrame
print(results_df.head(10))

# Save results
results_df['sentiment_score'] = results_df['Predicted Probabilities'].apply(lambda x: -1 * x[0] + 1 * x[2])
results_df.to_excel("scores.xlsx")

# Save model and tokenizer
model.save("sentimentModel_word2vec_lstm.keras")
with open('tokenizer_word2vec.pickle', 'wb') as handle:
    pickle.dump(token, handle, protocol=pickle.HIGHEST_PROTOCOL)

100/100 [==============================] - 3s 27ms/step - loss: 0.3136 - accuracy: 0.9071
Test Loss: 0.31360960006713867
Test Accuracy: 0.9071226716041565


2024-05-29 18:04:24.142072: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-05-29 18:04:24.143891: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-05-29 18:04:24.145199: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

100/100 [==============================] - 4s 31ms/step
Predicted probabilities: [[0.00186843 0.9879489  0.01018265]
 [0.00248631 0.07083724 0.92667645]
 [0.9840941  0.0100226  0.00588315]
 [0.00155931 0.98752207 0.01091862]
 [0.97370756 0.01863698 0.00765539]
 [0.00638556 0.9858951  0.00771951]
 [0.00246102 0.04474273 0.9527963 ]
 [0.00226001 0.9757517  0.02198827]
 [0.335406   0.06722655 0.59736735]
 [0.00185423 0.9877922  0.01035354]]
Actual labels: [[0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]
                                                Text  Actual Class  \
0  international leads consumer gainers nio tata ...             1   
1         pleased bjorn wahlroos accepted nomination             2   
2  us energy information administration projects ...             0   
3  edited transcript earnings conference call pre...             1   
4  okmetic expects net sales first half 2009 less...             0   
5 

In [16]:
results_df['sentiment_score'] = results_df['Predicted Probabilities'].apply(lambda x: -1 * x[0] + 1 * x[2])

In [17]:
results_df.to_excel("scores_lstm_word2vec.xlsx")